In [1]:
import os, math
from math import log, pow, sqrt

In [2]:
with open('query_list.txt', 'r') as fp:
    query_list = fp.read()
query_list = query_list.splitlines()

with open('doc_list.txt', 'r') as fp:
    doc_list = fp.read()
doc_list = doc_list.splitlines()

In [3]:
word_doc_num = {}
doc_word = {}
for doc in doc_list:
    with open('./docs/' + doc + '.txt', 'r') as fp:
        doc_content = fp.read()
    doc_content = doc_content.split(' ')
    doc_word[doc] = doc_content
    for word in doc_content:
        if word not in word_doc_num:
            word_doc_num[word] = {}
        if doc not in  word_doc_num[word]:
            word_doc_num[word][doc] = 0
        word_doc_num[word][doc] += 1
        
word_query_num = {}
query_word = {}
for query in query_list:
    with open('./queries/' + query + '.txt', 'r') as fp:
        query_content = fp.read()
    query_content = query_content.split(' ')
    query_word[query] = query_content
    for word in query_content:
        if word not in word_query_num:
            word_query_num[word] = {}
        if query not in  word_query_num[word]:
            word_query_num[word][query] = 0
        word_query_num[word][query] += 1

In [4]:
avg_doclen = 0
for doc in doc_list:
    avg_doclen += len(doc_word[doc])
avg_doclen = avg_doclen / len(doc_list)

In [5]:
# test
def get_sim(doc, query):
    query_content = query_word[query]
    doc_content = doc_word[doc]
    sim = 0
    
    b = 0.75
    k1 = 3
    s1 = k1 + 1
    k3 = 1
    s3 = k3 + 1  
    
    k2 = 0.1
    giq = sqrt(abs(k2 * len(query_content) * (avg_doclen - len(doc_content)) / (avg_doclen + len(doc_content))))
    
    for word in query_content:
        if word in doc_content:
            tfij = word_doc_num[word][doc]
            tfiq = word_query_num[word][query]
            log_thing = log((len(doc_list) - len(word_doc_num[word]) + 0.5) / (len(word_doc_num[word]) + 0.5))
            up = (k1 + 1) * tfij * (k3 + 1) * tfiq * pow(log_thing, 1.3) 
            down = (k1*((1 - b) + (b * len(doc_content) / avg_doclen)) + tfij) * (k3 + tfiq)
            sim += giq + (up / down) ** 1.19
    return sim

In [6]:
# best
def get_sim(doc, query):
    query_content = query_word[query]
    doc_content = doc_word[doc]
    sim = 0
    
    b = 0.87
    k1 = 3
    s1 = k1 + 1
    k3 = 1
    s3 = k3 + 1  
    
    k2 = 0.1
    giq = sqrt(abs(k2 * len(query_content) * (avg_doclen - len(doc_content)) / (avg_doclen + len(doc_content))))
    
    for word in query_content:
        if word in doc_content:
            tfij = word_doc_num[word][doc]
            tfiq = word_query_num[word][query]
            log_thing = log((len(doc_list) - len(word_doc_num[word]) + 0.5) / (len(word_doc_num[word]) + 0.5))
            up = (k1 + 1) * tfij * (k3 + 1) * tfiq * pow(log_thing, 1.3) 
            down = (k1*((1 - b) + (b * len(doc_content) / avg_doclen)) + tfij) * (k3 + tfiq)
            sim += giq + (up / down) ** 1.19
    return sim

In [13]:
result = {}
for query in query_list:
    for doc in doc_list:
        if query not in result:
            result[query] = {}
        result[query][doc] = get_sim(doc, query)

In [14]:
f = open('result.txt', 'w+')
f.write('Query,RetrievedDocuments\n')

for query in query_list:
    print_ans = query + ','
    temp_result = sorted(result[query].items(), key=lambda x:x[1],reverse=True)
    for i in temp_result:
        print_ans += i[0] + ' '
    print_ans = print_ans[:-1] + '\n'
    f.write(print_ans)

f.close()

In [8]:
sorted(result['301'].items(), key=lambda x:x[1],reverse=True)

[('FBIS3-23986', 25.33733111427248),
 ('FBIS4-7811', 25.108555497620966),
 ('FBIS3-19646', 24.442362104189208),
 ('FBIS3-21961', 24.442362104189208),
 ('FBIS4-43965', 23.83022923717901),
 ('FBIS4-68801', 23.660581458524685),
 ('FBIS3-55219', 23.622953613629868),
 ('FBIS3-19199', 23.489553041930098),
 ('FBIS3-26415', 23.059903063730967),
 ('FBIS3-41247', 23.059903063730967),
 ('FBIS3-58028', 22.444812246200456),
 ('FBIS3-58058', 22.444812246200456),
 ('FBIS3-60984', 22.261764511906442),
 ('FBIS4-38364', 22.23010657780195),
 ('FBIS3-38070', 22.057234746596166),
 ('FBIS4-46846', 21.93802322055197),
 ('LA032790-0103', 21.92974873334359),
 ('FBIS3-15586', 21.914884645523088),
 ('FBIS3-58055', 21.886054142242102),
 ('FBIS4-45477', 21.538352420404777),
 ('FBIS4-43801', 21.512583334740583),
 ('FBIS3-21770', 21.243532526394194),
 ('FBIS4-54904', 20.954791125472497),
 ('FBIS4-34836', 20.909061955513),
 ('FBIS4-46780', 20.877980142665812),
 ('FBIS3-19420', 20.81438244800381),
 ('LA040489-0112', 2

In [ ]:
# 用一些怪方法，然後現在是最好的 = =

In [9]:
total_query = []
for word, other in word_query_num.items():
    if word not in total_query:
        total_query.append(word)

In [10]:
temp_word_doc_num = {}
temp_doc_word = {}

for word, other in word_doc_num.items():
    if word in total_query:
        temp_word_doc_num[word] = other
        
for doc, other in doc_word.items():
    temp_doc_word[doc] = []
    for word in other:
        if word in total_query:
            temp_doc_word[doc].append(word)

In [11]:
temp_avg_doclen = 0
for doc in doc_list:
    temp_avg_doclen += len(temp_doc_word[doc])
temp_avg_doclen = temp_avg_doclen / len(doc_list)

In [12]:
def get_sim(doc, query):
    query_content = query_word[query]
    doc_content = temp_doc_word[doc]
    sim = 0
    
    b = 0.87
    k1 = 3
    s1 = k1 + 1
    k3 = 1
    s3 = k3 + 1  
    
    k2 = 0.1
    giq = sqrt(abs(k2 * len(query_content) * (temp_avg_doclen - len(doc_content)) / (temp_avg_doclen + len(doc_content))))
    
    for word in query_content:
        if word in doc_content:
            tfij = temp_word_doc_num[word][doc]
            tfiq = word_query_num[word][query]
            log_thing = log((len(doc_list) - len(temp_word_doc_num[word]) + 0.5) / (len(temp_word_doc_num[word]) + 0.5))
            up = (k1 + 1) * tfij * (k3 + 1) * tfiq * pow(log_thing, 1.3) 
            down = (k1*((1 - b) + (b * len(doc_content) / temp_avg_doclen)) + tfij) * (k3 + tfiq)
            sim += giq + (up / down) ** 1.19
    return sim